In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

In [2]:
# Carga de datos
df = pd.read_csv('/content/P6_datos.csv', sep = ';', index_col = 0)

# Análisis de consistencia
df.drop(labels = ['8','9', '14', '15', '17'], axis = 1, inplace = True)

# Altura
df.index = [0.4, 1., 2., .2, .8, 1.42]

#### Estimar velocidad para 0.2, 0.8 y 1.42

In [3]:
a = []; b = []
for i in range(len(df.columns)):
  dvz = df.iloc[:3,i]
  ab = np.polyfit(x = np.log(list(dvz.index.values/2)), 
                  y = np.log(list(dvz.values)), deg = 1)
  b.append(ab[0]); a.append(ab[1])

ab_t = pd.DataFrame({'a': a, 'b': b}, index = df.columns).transpose()
df = pd.concat([df,ab_t], axis=0)

In [4]:
ab_t

,10,11,12,13,16
a,1.102906,0.797998,0.925531,0.639961,-0.289555
b,0.913386,0.747514,0.665345,0.834013,1.337277


In [5]:
V_20cm = np.e**(df.loc['a'] + df.loc['b']*0.2)
V_80cm = np.e**(df.loc['a'] + df.loc['b']*0.8)
V_142cm = np.e**(df.loc['a'] + df.loc['b']*1.42)

v_est = pd.DataFrame({0.2: V_20cm, 0.8: V_80cm, 1.42: V_142cm}, 
                     index = df.columns).transpose()
v_est

,10,11,12,13,16
0.20,3.616778,2.579256,2.882333,2.240644,0.978143
0.80,6.256463,4.039049,4.296527,3.695705,2.182055
1.42,11.022232,6.420321,6.490378,6.198205,4.999703


#### Número de Reynolds (Ri)

In [6]:
v_80_20 = (v_est.loc[.8] - v_est.loc[.2])/(0.8-0.2)
v_142_80 = (v_est.loc[1.42] - v_est.loc[.8])/(1.42-0.8)
du_dz = pd.DataFrame({'duz1': v_80_20, 'duz2': v_142_80},
                     index = df.columns).transpose()
du_dz

,10,11,12,13,16
duz1,4.399476,2.432989,2.35699,2.425102,2.006519
duz2,7.686723,3.840760,3.53847,4.036290,4.544595


In [7]:
gvt1 = (df.iloc[4] - df.iloc[3])/(0.8-0.2)
gvt2 = (df.iloc[5] - df.iloc[4])/(0.8-0.2)
t_p1 = (df.iloc[4]+273.15 + df.iloc[3]+273.15)/2
t_p2 = (df.iloc[5]+273.15 + df.iloc[4]+273.15)/2

Ri_80_20 = 9.8/t_p1*(0.01 - gvt1)/(v_80_20**2)
Ri_142_80= 9.8/t_p2*(0.01 - gvt2)/(v_142_80**2)

Ri = pd.DataFrame({'80 - 20 cm': Ri_80_20, '142 - 80 cm': Ri_142_80},
                     index = df.columns).transpose()
Ri

,10,11,12,13,16
80 - 20 cm,0.000855,0.004597,0.005846,0.009153,0.004065
142 - 80 cm,0.000280,0.001117,0.000883,0.001337,0.000793


#### df

In [ ]:
df

,10,11,12,13,16
0.40,0.68,0.63,0.83,0.45,0.08
1.00,1.67,1.51,1.75,1.33,0.36
2.00,2.94,2.06,2.39,1.67,0.67
0.20,28.90,30.80,31.90,32.60,32.40
0.80,28.60,30.30,31.30,31.60,32.10
1.42,28.30,30.00,31.10,31.20,31.80
